<a href="https://colab.research.google.com/github/annykay/scientificComputations/blob/main/SciComp2021_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scientific Computing 2021: Homework Assignment 1 
Due Monday October 11, 2021, 23:59

### Problem 1 (2 points)
Under assumptions of Amdahl's law, suppose that 75% of a program are perfectly parallelizable, and the rest is not parallelizable. 
1. What is the maximum speedup achievable by parallelization? 
2. Suppose that we have obtained speedup 2 (by using a suitable number of processes). What is the efficiency of this parallelization?

**Problem 1, solution**

1) The maximun  speedup, that could be achieved is 4 times, due to Amdahl's law. We have 0.75 part of code that can be parallelized, so the speedup = $\frac{1}{1-0.75} = 4$

2) speedup = $\frac{1}{(\alpha + \frac{1 - \alpha}{p})}$

speedup = 2;

$\alpha = 0.75$ 

=> p = 3

efficacy = 2/3




### Problem 2 (2 points)
Write a Python or C/C++ program that uses **MPI reduce** to find the largest file in terms of the  number of lines among all .txt files in the working directory (only .txt files should be examined). The program must be callable in the form `mpirun -np <N> python linecount.py` (in the case of the Python version) or `mpirun -np <N> linecount.exe` (the C/C++ version), where `<N>` is the user-defined number of processes. The program is expected to first somehow distribute the files found in the current directory to the processes, then each process should count the lines in the files assigned to it, and finally the result should be MPI-reduced and printed out. The program only needs to output the number of lines in the largest file; no need to output the name of this file. The program must work correctly even if the number of files is not divisible by the number of processes.

**Problem 2, solution**

See the solution in the file prog.py. The program can be executed by the command `mpirun -n <N> python prog.py`, where `<N>` - is the number of processes 

In [ ]:
import numpy as np

def invPerm(p):
    '''Invert the permutation p'''
    s = np.empty(p.size, p.dtype)
    s[p] = np.arange(p.size)
    return s


def getSA(A):
    if not type(A) is np.ndarray:
        A = np.array(list(A))
    N = len(A) 
    M = int(np.ceil(np.log2(N)))+1   # number of iterations
    
    # auxiliary arrays; row m stores results after m'th step:
    
    # positions of sorted length-(2**m) sequences in A
    P = np.zeros((M,N), dtype=int) 
    
    # rank (0, 1, etc.) of sorted length-(2**m) sequences after sorting
    Q = np.zeros((M,N), dtype=int)     
    
    # rank of sorted length-(2**m) sequences at its starting position in A;
    # padded by 0 on the right
    R = np.zeros((M,3*N), dtype=int) 

    for k in range(M):
        if k == 0:
            P[0] = np.argsort(A)
            Q[0][1:] = np.cumsum(A[P[0]][1:] != A[P[0]][:-1])
            R[0][:N] = Q[0][invPerm(P[0])]
        else:
            offset = 2**(k-1)
            r = np.lexsort((R[k-1, P[k-1]+offset], R[k-1, P[k-1]]))
            P[k] = P[k-1][r]
            # k'th rank increases iff (k-1)'th rank increases at least for one element of the pair    
            Q[k][1:] = np.cumsum(np.logical_or(R[k-1][P[k]][1:] != R[k-1][P[k]][:-1], 
                                          R[k-1][P[k]+offset][1:] != R[k-1][P[k]+offset][:-1]))
            R[k][:N] = Q[k][invPerm(P[k])]
            
            # early stopping if suffixes already fully sorted (max rank is N-1)
            if Q[k][-1] == N-1: 
                break
    
    SA = P[k]
    return SA, P[:k+1], Q[:k+1], R[:k+1] 



def getLCP(SA, R):
    (M, N) = R.shape
    LCP = np.zeros((len(SA)-1,),dtype=int)
    for m in range(M-1)[::-1]:
        t = (R[m][SA[1:]+LCP] == R[m][SA[:-1]+LCP]).astype(int)
        LCP += (2**m)*t
    return LCP

def getNumString(A):
    N = len(A)
    SA, _, _, R = getSA(A)
    LCP = getLCP(SA, R)
    return int(N*(N-1)/2 - sum(LCP) )


def main():
  n = int(input())
  for i in range(n):
    string = input()
    string += '$'
    print(getNumString(string))


if __name__ == '__main__':
    main()          

2
ccccc
5
ABABA
9


### Problem 3 (2 points)
Solve the Distinct Substrings problem at Sphere online judge: http://www.spoj.com/problems/DISUBSTR/. Provide code passing the test of the judge. Explain how your code works and theoretically estimate the complexity of the algorithm (as $O(f(N))$, where $f(N)$ is some function of the length of the input string). 

**Problem 3, solution**

Here is the fuction, that solves the third program. The main idea of the program is: if we calculate all unique prefixis of all suffixies of the string, we will find the amount of substrings. So we calculate the suffix array of the input string, then count the overlap of neighboring suffixies(LCP). Each suffix can have L prefixes (L = $length(suffix_i)$), but it has LCP[i] common prefixies with previous one. So we just have to sum lengths of all suffixies and subtract the LCP values(intersactions). 

Constracting Suffix Array: $O(N\log^2(N))$

Calculating LCP: $O(N\log(N))$

Summation: O(N)

As the complexity of the algotythm is determined by its longest part, the total complexity of my code will be $O(N\log^2(N))$, where N - is the number of a string. If we have M strings, the total comlexity will be $O(M*N\log^2(N))$

In [ ]:
!pip install mpi4py

     |████████████████████████████████| 2.4 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.1-cp37-cp37m-linux_x86_64.whl size=2180584 sha256=d79bfa26fd479411c18287423b565700096e04e71ab8d343abbb9b20d3240fdb
  Stored in directory: /root/.cache/pip/wheels/91/be/c0/2b0347be1de5cd8ca9fe67da7ec8c3fe8930fcb6b0df6f2255
Successfully built mpi4py


In [ ]:
!mpirun --allow-run-as-root -n 7 python prog.py 

Max size: 67



### Problem 4 (2 points)
Suppose that we want to distribute $N$ personal projects to $N$ students. Assume that each student $(k)_{k=0}^{N-1}$ has a list of his/her preferences for the projects, expressed as a vector $\mathbf r_k$ of integer ranks assigned to each project. Ranks vary between 0 and $N-1$ without repetitions, the **lower** the rank the **more preferable** the project. (For example, the first student's ranks are $\mathbf r_0 = [2,1,0]$, the second's $\mathbf r_1 = [0,2,1]$ and the third $\mathbf r_2 = [0,1,2]$). We want to distribute the projects so as to maximize the total preference, i.e., if $n_k$ denotes the project assigned to the $k$'th student, we want to make $f = \sum_{k=0}^{N-1} \mathbf r_k[n_k]$ as small as possible. (In the example above the optimal distribution is $n_0=2, n_1=0, n_2=1$, which gives $f=1$).  
  * Come up with an algorithm optimizing the distribution and implement it in a Python or C/C++ program. The algorithm should accept the preference vectors and output a recommended distribution $(n_k)_{k=1}^N$. The algorithm need not find the best solution, but is expected to generally produce better solutions than would have been obtained by randomly distributing the projects. The algorithm should be reasonably fast, say run in not more than a few seconds for $N=30$. 
  * Compare experimentally your algorithm with the trivial algorithm producing a random distribution. To this end, perform $M=1000$ experiments in each of which 1) random preference vectors for $N=30$ students and projects are generated; 2) the objective function $f$ is evaluated for both algorithms. After finishing all the experiments, plot the two respective distributions of the obtained $M$ values of $f$ and compute the mean values of $f$ for both algorithms. 

**Problem 4, Solution**

The idea of the algorythm is easy. Every student can be represented as a vector of ranks for different projects, so all students can be represented as a matrix.

After that we can find the "worst" project: it will have tha maximum sum of rangs among all students. And we assign this project to the "best" student for this project: the one that has the minimum rang for this project. 

Then we delete the raw and the column of selected project and student respectivly. 

Then we calculate the sum of rangs of projects for each student, find the "worst" student (with maximum sum of ranks), and after that find for him the "best" project(with minimum rank). We assign this "best" project to the "worst" one and delete the raw and the column of selected project and student respectivly. 

We repeat this algo while we have elements in the matrix. And in the end we have a dictionary for projects and students.

The code is shown below. 

In [96]:
import numpy as np
import random
import timeit

#read the number of students and projects
N = int(input())
#generate a random matrix of projects for students
projects = [i for i in range(N)]
students = np.array([random.sample(projects, N) for i in range(N)])
students1 = students

30


In [97]:

start = timeit.default_timer()

#initial values for supporting elements
finalProj = {}
keysProj = [i for i in range(N)]
#print('keysProj:', keysProj)
#valsProj = [i for i in range(N)]
keysStud = [i for i in range(N)]
#print('keysStud:', keysStud)
#valsStud = [i for i in range(N)]


while len(students) > 0:
  #print('length:', len(students))

  #finding the "worst" project
  sums = np.sum(students, axis = 0)
  #print('sums:', sums)
  worseProjs = np.where(sums == np.amax(sums))
  #print('worseProjs:', worseProjs)
  worseProj = random.sample(list(worseProjs[0]), 1)[0]

  #find the "best" student for this project
  #print('worseProj:', worseProj)
  bestStudents = np.where( students[:, worseProj] == np.min(students[:, worseProj]))
  #print('bestStudents:', bestStudents)
  bestStudent = random.sample(list(bestStudents[0]), 1)[0]
  #print('bestStudent:', bestStudent)

  #delete rows and colums, get right values 
  keyBestStudent = np.where(keysStud == bestStudent)[0][0]
  #print('keyBestStudent:', keyBestStudent)
  keyWorstProj = np.where(keysProj== worseProj)[0][0]
  #print('keyWorstProj:', keyWorstProj)
  finalProj[keyBestStudent] = keyWorstProj
  #print('finalProj:', finalProj)
  #print(keyBestStudent,keyWorstProj)

  students = np.delete(students, (bestStudent), axis=0)
  #print('students:', students)
  students = np.delete(students, (worseProj), axis=1)
  #print('students:', students)

  if keysStud[bestStudent] > 0:
    keysStud[bestStudent] *= -1
  else:
    keysStud[bestStudent] = -10000
  #print('keysStud:', keysStud)
  if keysProj[worseProj] > 0:
    keysProj[worseProj]  *= -1
  else:
    keysProj[worseProj]  = -10000
  #print('keysProj:', keysProj)
  
  for i in range(bestStudent+1, N): 
    if (keysStud[i]) > 0:
      keysStud[i] -= 1
  #print('keysStud:', keysStud)
  for i in range(worseProj+1, N): 
    if (keysProj[i] > 0 ):
      keysProj[i] -= 1
  #print('keysProj:', keysProj)
  
  #check if number of elements of is not zero
  if len(students) > 0:
    #find the "worst" student 
    sums = np.sum(students, axis = 1)
    #print('sums:', sums)
    worseStudents = np.where(sums == np.amax(sums))
    #print('worseStudents:', worseStudents)
    worseStudent = random.sample(list(worseStudents[0]), 1)[0]
    #print('worseStudent:', worseStudent)

    #finding the "best project for him"
    bestProjs = np.where( students[:, worseStudent] == np.min(students[:, worseStudent]))
    #print('bestProjs:', bestProjs)
    bestProj = random.sample(list(bestProjs[0]), 1)[0]
    #print('bestProj:', bestProj)

    #delete row, column and change keys
    keyBestProj = np.where(keysProj == bestProj)[0][0]
    #print('keyBestProj:', keyBestProj)
    keyWorstStudent = np.where(keysStud == worseStudent)[0][0]
    #print('keyWorstStudent:', keyWorstStudent)
    finalProj[keyWorstStudent] = keyBestProj
    students = np.delete(students, (worseStudent), axis=0)
    #print('students:', students)
    students = np.delete(students, (bestProj), axis=1)
    #print('students:', students)


    if keysStud[worseStudent] > 0:
      keysStud[worseStudent] *= -1
    else:
      keysStud[worseStudent] = -10000
    #print('keysStud:', keysStud)
    if keysProj[bestProj] > 0:
      keysProj[bestProj]  *= -1
    else:
      keysProj[bestProj]  = -10000
    #print('keysProj:', keysProj)

    for i in range(worseStudent+1, N): 
      if (keysStud[i]) > 0:
        keysStud[i] -= 1
    #print('keysStud:', keysStud)
    for i in range(bestProj+1, N): 
      if (keysProj[i] > 0 ):
        keysProj[i] -= 1
    #print('keysProj:', keysProj)

#find the last pair student:project
for i in range(N):
  if i not in finalProj.keys():
    lastStudent = i
    break
for i in range(N):
  if i not in finalProj.values():
    lastProj = i
    break
finalProj[lastStudent] = lastProj


stop = timeit.default_timer()

print('Time of the algorythm: ', stop - start, 's')  

Time:  0.003531199999997625


Here you can see, that the time of the execution for N=3 is not more than a second, so the algo is quite fast

In [98]:
result = 0
for (i,j) in finalProj.items():
  result += students1[i][j]
print('The sum of ranks of assigned projects:', result)

In [99]:
#check with random values

#generate random values
M = 30
randomProj = [random.sample(projects, N) for i in range(M)]

resultsRand = [0 for j in range(M)]

#calculate the sum of ranks of randomly assigned projects
for j in range(M):
  for i in range(N):
    resultsRand[j]+= students1[i][randomProj[i][j]]

#compare it with the result of algo

print('Cases where random distribution is better:', sum(resultsRand < result))

228

  
### Problem 5 (2 points)
Suppose that we have developed an algorithm that is supposed to generate independent (quasi-)random numbers uniformly distributed in the interval $[0,1]$. To test our algorithm, we perform a series of experiments. In each experiment, we generate $N=10^3$ numbers $(x_n)_{n=1}^N$ with our algorithm, and compute the minimum distance $r=\min_{1 \le n < m\le N}|x_n-x_m|$ between them. We observe that in more than 99% of such experiments we obtain $r < 10^{-5}$. Does this observation contradict the hypothesis of generating independent uniformly distributed random numbers? Explain your answer.

**Task 5**

Let's calculate the probability of such an event:

$P(\min|\xi_i - \zeta_j| < r; i = \overline{1,n}; j = \overline{1,n}) = 1 - P(\min|\xi_i -\zeta_j| \geq r) = [|\xi_i - \zeta_j| = \phi_k; k = \overline{1, N = \frac{n(n-1)}{2}}] = 1 - P(min (\phi_k) \geq r) = 1 -  \prod_{k=1}^{N} P(\phi_k \geq r) = 1 - (1 - P(\phi_k < r ))^N = [P(\phi_k < r ) = \Phi(r)] = 1 - (1 - \Phi(r)))^N $

Now let's estimate $\Phi(r)$ using the geometry probability. 

$|\xi_i - \zeta_j| < x $

$ - x <\xi_i - \zeta_j < x$

$ - x + \zeta_j<\xi_i  < x + \zeta_j$

As we know that $\zeta_j, \xi_i$ lie in thhe interval [0,1], we can estimate the $\Phi(x)$ as a square of intersation of a unit quadrate and lines  $\xi_i = - x + \zeta_j; \xi_i= x + \zeta_j$. It will be:


\begin{equation*}
 \Phi(x) =
 \begin{cases}
   0, x \leq 0  
   \\
   1 - \frac{2*(1-x)^2}{2} = 2x - x^2 , 0 < x < 1
   \\
   1, x \geq 1
 \end{cases}
\end{equation*}

add it to our previous results: 
$1 - (1 - \Phi(r)))^N = 1 - (1 - 2r + r^2)^N = 1 - (1 - r)^{2N} = 1 - (1 - 10^{-5})^{10^6} = [python] \approx 0.9999546023401903  $ 

And that is actually what we see in our simmulation, so everything is okey with the generator. 


ADD A PICTURE 


In [ ]:
1 - (1-10**(-5))**(1000*1000)


0.9999546023401903